In [1]:

import numpy as np
import pandas as pd
import json
import os
import wikipediaapi
import re

import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import wikipediaapi
from tqdm.autonotebook import tqdm

C:\Users\gabeb\AppData\Local\Temp\ipykernel_25504\2933910628.py:16: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
# CHROME_BINARY_LOCATION = "/usr/bin/chrome-linux64/chrome"
# CHROMEDRIVER_BINARY_LOCATION = "/usr/bin/chromedriver-linux64/chromedriver"
chromedriver_path = 'C:/Users/gabeb/Desktop/programming/chromedriver-win64/chromedriver.exe'

def add_driver_options(options):
    """
    Add configurable options
    """
    chrome_options = Options()
    for opt in options:
        chrome_options.add_argument(opt)
    return chrome_options

def initialize_driver():
    """
    Initialize the web driverpip 
    """
    driver_config = {
        "options": [
            "--headless",
            "--no-sandbox",
            "--start-fullscreen",
            "--allow-insecure-localhost",
            "--disable-dev-shm-usage",
            "user-agent=Chrome/116.0.5845.96"
        ],
    }
    options = add_driver_options(driver_config["options"])
    # options.binary_location = CHROME_BINARY_LOCATION
    driver = webdriver.Chrome(
        executable_path=chromedriver_path,
        options=options)
    return driver

In [3]:
def parse_rollercoaster_page(soup):
    # Initialize variables to store information
    city = region = country = make = None
    trackStats = {}
    image_urls = []

    # Find location information
    feature = soup.find('div', {"id": "feature"})
    if feature:
        descList = feature.findAll('div')
        if descList:
            desc = descList[0]
            locationLinks = desc.findAll('a')
            if len(locationLinks) == 3: # amusement park, city, country
                city = locationLinks[1].contents[0]
                region = locationLinks[1].contents[0]  # Assuming region is the same as city
                country = locationLinks[2].contents[0]
            else:
                city = locationLinks[-3].contents[0]
                region = locationLinks[-2].contents[0]
                country = locationLinks[-1].contents[0]

            # Find make information
            if len(descList) >= 2:
                make_link = descList[1].find('a')
                if make_link:
                    make = make_link.contents[0]

    # Find track information
    trackStatsTable = soup.find('table', {'class': 'stat-tbl'})
    if trackStatsTable:
        trackStatsTag = trackStatsTable.findAll('tr')
        for tr in trackStatsTag:
            key = tr.find('th').text.strip()
            valueTag = tr.find('td')
            if valueTag:
                value = valueTag.text.strip()
                if key == "Elements": # list of links
                    value = [a.contents[0] for a in valueTag.findAll('a')]
                elif valueTag.find('span', {'class': 'float'}) is not None: # float value
                    unit = valueTag.contents[1].strip() if len(valueTag.contents) >= 2 else ""
                    key = f"{key} ({unit})"
                    value = valueTag.find('span', {'class': 'float'}).text.strip()
            else:
                value = None
            trackStats[key] = value

    # Find images
    image_container = soup.find('a', {'id': 'opfanchor'})
    if image_container:
        image_tags = image_container.findAll('img')
        for img in image_tags:
            if img.has_attr('data-original'):  # Check if 'data-original' attribute exists
                image_url = urljoin(rcdb_url, img['data-original'])  # Use 'data-original' attribute if available
            else:
                image_url = urljoin(rcdb_url, img['src'])  # Otherwise, fallback to 'src' attribute
            image_urls.append(image_url)

    return city, region, country, make, trackStats, image_urls

def parse_rollercoaster_table_row(row):
    # Get data from the table row
    elements = row.findAll('td')
    coasterName = elements[1].find('a').text.strip()
    coasterLink = elements[1].find('a')['href']
    amusementPark = elements[2].find('a').text.strip()
    coasterType = elements[3].find('a').text.strip()
    coasterDesign = elements[4].find('a').text.strip()
    statusTag = elements[5].find('a')
    status = statusTag.contents[0] if statusTag else "Removed"
    openedTag = elements[6].find('time')
    opened = openedTag.contents[0] if openedTag else "unknown"

    # Get data from the coaster specific link
    driver.get(f"{rcdb_url}{coasterLink}")
    coaster_soup = BeautifulSoup(driver.page_source, 'html')
    city, region, country, make, trackStats, image_urls = parse_rollercoaster_page(coaster_soup)

    return {
        'Name': coasterName, 
        'Amusement Park': amusementPark, 
        'Type': coasterType, 
        'Design': coasterDesign, 
        'Status': status, 
        'Opened': opened, 
        'City': city, 
        'Region': region, 
        'Country': country, 
        'Make': make,
        **trackStats,
        'Image URLs': image_urls
    }


In [4]:
rcdb_url = "https://rcdb.com"

# Path to your ChromeDriver executable
chromedriver_path = r'C:\Users\gabeb\Desktop\programming\chromedriver-win64\chromedriver.exe'

# Initialize Chrome WebDriver
# service = Service(chromedriver_path)
# service.start()
driver = webdriver.Chrome()
# driver = initialize_driver()
driver.get(f"{rcdb_url}/r.htm?ot=2")
soup = BeautifulSoup(driver.page_source, 'html')
numPages = int(soup.find('div', {"id": "rfoot"}).findAll('a')[-2].contents[0])
print(f"Total number of pages to iterate through: {numPages}")

pbar = tqdm(range(numPages))
rollercoasters = []

for pageNum in pbar:
    print(f"Getting information for page {pageNum+1}")
    driver.get(f"{rcdb_url}/r.htm?page={pageNum+1}&ot=2")
    soup = BeautifulSoup(driver.page_source, 'html')
    table = soup.findAll('table')[1].find('tbody')
    rows = table.findAll('tr')
    
    for row in rows:
        rowData = parse_rollercoaster_table_row(row)
        pbar.set_postfix(
            coasterName=rowData['Name'], 
            amusementPark=rowData['Amusement Park'])
        rollercoasters.append(rowData)
        
    # every page, save the information to a dataframe
    df = pd.DataFrame.from_records(rollercoasters)
    df.to_csv("data.csv", index=False)

Total number of pages to iterate through: 498


  0%|          | 0/498 [00:00<?, ?it/s]

Getting information for page 1


  0%|          | 1/498 [00:03<27:06,  3.27s/it, amusementPark=Parque de Atracciones de Madrid, coasterName=Abismo]            

Getting information for page 2


  0%|          | 1/498 [00:04<36:23,  4.39s/it, amusementPark=Speelpark Oud Valkeveen, coasterName=Achtbaan]      


TypeError: object of type 'NoneType' has no len()

In [ ]:
driver.quit()

In [ ]:
df = pd.read_csv('data.csv')
df.head()

,Name,Amusement Park,Type,Design,Status,Opened,City,Region,Country,Make,...,Duration,G-Force (),Vertical Angle (°),Uphill Length (ft),Downhill Length (ft),Δ Elevation (ft),Airtime Points,Crossings,Bank Angle (°),Drop
0,€uro-Coaster,Wiener Prater,Steel,Suspended,Removed,5/29/2020,Vienna,Vienna,Austria,Reverchon,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,€uro Coaster,Funland Theme Park,Steel,Sit Down,Removed,2021,Somerset,England,United Kingdom,SBF Visa Group,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1066,Festyland,Steel,Sit Down,Operating,3/27/2005,Bretteville-Sur-Odon,Normandy,France,Soquet,...,1:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10 Inversion Roller Coaster,Chimelong Paradise,Steel,Sit Down,Operating,2/2006,Guangzhou,Guangdong,China,Intamin Amusement Rides,...,1:32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1970 Galaxy Rip Tide Coaster,Swampy Jack's Wongo Adventure,Steel,Sit Down,Relocated,6/27/2014,Panama City Beach,Florida,United States,S.D.C.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:

# df = df[df['Name'] != 'unknown']
# wiki_wiki = wikipediaapi.Wikipedia(
#     user_agent='MyProjectName (gabe@example.com)',
#         language='en',
#         extract_format=wikipediaapi.ExtractFormat.WIKI
# )
# def fetch_ride_history(ride_name, amusement_park):
#     try:
#         # Search Wikipedia for the page of the rolle|r coaster
#         query = f"{ride_name} {(amusement_park)}"
#         page = wiki_wiki.page(query)
#         print(page.text)
#         # Find the section titled "History" and extract its text
#         history_text = ""
#         if page.exists():
#             sections = page.sections
#             for section in sections:
#                 if section.title.lower() == 'history':
#                     history_text = section.text
#                     break
#             # Remove any citation numbers (e.g., [1], [2], etc.)
#             history_text = re.sub(r'\[\d+\]', '', history_text)
#         return history_text
#     except Exception as e:
#         print(f"Error fetching history for {ride_name} at {amusement_park}: {e}")
#         return "History not found"
    
# fetch_ride_history('Batman: the Ride',"(Six Flags México)")
# def add_history_to_dataframe(df):
#     # Create a new column to store the history
#     df['History'] = ""
#     # Iterate through each row in the DataFrame
#     for index, row in df.iterrows():
#         ride_name = row['Name']
#         amusement_park = row['Amusement Park']
#         # Fetch history from Wikipedia
#         history = fetch_ride_history(ride_name, amusement_park)
#         # Update the 'History' column with the retrieved history
#         df.at[index, 'History'] = history
#     return df

# # Add history to DataFrame
# df_with_history = add_history_to_dataframe(df)

# # Save the updated DataFrame to the same CSV file (overwrite)
# df_with_history.to_csv("data_with_history.csv", index=False)


KeyboardInterrupt: 

In [ ]:
wiki_wiki = wikipediaapi.Wikipedia(
    user_agent='MyProjectName (gabe@example.com)',
        language='en',
        extract_format=wikipediaapi.ExtractFormat.WIKI
)
p_wiki = wiki_wiki.page("Switchback Railway")
print(p_wiki.text)
section_history = p_wiki.section_by_title('History')
# print((section_history.text))


The original Switchback Railway was the first roller coaster  at Coney Island in Brooklyn, New York City, and one of the earliest designed for amusement in the United States. The 1885 patent states the invention relates to the gravity double track switchback railway, which had predicated the inclined plane railway, patented in 1878 by Richard Knudsen.  Coney Island's version was designed by LaMarcus Adna Thompson in 1881 and constructed in 1884. It appears Thompson based his design, at least in part, on the Mauch Chunk Switchback Railway which was a coal-mining train that had started carrying passengers as a thrill ride in 1827.For five cents, riders would climb a tower to board the large bench-like car and were pushed off to coast 600 ft (183 m) down the track to another tower. The car went just over 6 mph (9.7 km/h). At the top of the other tower the vehicle was switched to a return track or "switched back" (hence the name).
This track design was soon replaced with an oval complete-c

AttributeError: 'NoneType' object has no attribute 'text'

In [ ]:
def fetch_ride_history(coaster_name):
    wiki_wiki = wikipediaapi.Wikipedia(
        user_agent='MyProjectName (gabe@example.com)',
        language='en',
        extract_format=wikipediaapi.ExtractFormat.WIKI
    )
    page = wiki_wiki.page(coaster_name)
    if not page.exists():
        return "History not found"
    section_history = page.section_by_title('History')
    if section_history:
        return section_history.text
    else:
        return "History section not found"

def parse_coaster_infobox(url, coaster_name, filter_cols=False):
    try:
        df = pd.read_html(url, attrs={'class': 'infobox'})[0]
    except ValueError:
        return pd.DataFrame()

    mycoaster = df.columns[0]
    df = df[df.columns[:2]]
    df.columns = ['Coaster', 'Value']
    df = df.set_index('Coaster')
    df = df.loc[~df.index.isna()]
    if filter_cols:
        MAIN_KEYS = ['coaster_name', 'Length', 'Speed', 'Location', 'Coordinates',
                     'Status', 'Opening date',
                     'General statistics', 'Type', 'Manufacturer',
                     'Height restriction',
                     'Model', 'Height', 'Inversions', 'Lift/launch system',
                     'Cost', 'Trains',
                     'Park section', 'Duration', 'Capacity',
                     'G-force', 'Designer',
                     'Max vertical angle', 'Drop', 'Soft opening date',
                     'Fast Lane available', 'Replaced', 'Track layout',
                     'Fastrack available','Soft opening date',
                     'Closing date', 'Replaced by', 'General Statistics',
                     'Website', 'Flash Pass Available', 'Must transfer from wheelchair',
                     'Theme', 'Single rider line available',
                     'Restraint Style',
                      'Flash Pass available', 'Acceleration',
                      'Restraints', 'Name','year_introduced']

        FILTERED_KEYS = [v for v in df.index if v in MAIN_KEYS]
        df = df.loc[FILTERED_KEYS]
    df = df.T
    df.index = [mycoaster]
    df = dedup_infobox(df)
    df['coaster_name'] = coaster_name
    
    # Fetch history from Wikipedia
    history = fetch_ride_history(coaster_name)
    df['History'] = history
    
    return df

def dedup_infobox(df):
    dup_cols = df.columns[df.columns.duplicated()].unique()
    for c in dup_cols:
        cols = []
        count = 1
        for column in df.columns:
            if column == c:
                cols.append(f'{c}_{count}')
                count += 1
                continue
            cols.append(column)
        df.columns = cols
    return df

DEBUG = False
wiki = wikipediaapi.Wikipedia(
    user_agent='MyProjectName (gabe@example.com)',
    language='en',
    extract_format=wikipediaapi.ExtractFormat.WIKI
)
cat = wiki.page('Category:Roller_coaster_introductions_by_year')
i = 0
ds = []
for year in tqdm(cat.categorymembers):
    cat_year = wiki.page(year)
    cat_year.categorymembers
    for coaster, page in cat_year.categorymembers.items():
        url = page.fullurl
        parsed_df = parse_coaster_infobox(url, coaster, filter_cols=True)
        parsed_df['year_introduced'] = int(year[-4:])
        ds.append(parsed_df)
        i += 1
    if i == 20 and DEBUG:
        break
if not DEBUG:
    coaster_db = pd.concat(ds)

# Save the DataFrame to CSV
coaster_db.to_csv('coaster_db_v2.csv')


100%|██████████| 108/108 [13:42<00:00,  7.62s/it]


In [ ]:
import pandas as pd

# Load the first CSV file
df1 = pd.read_csv('data.csv')

# Load the second CSV file
df2 = pd.read_csv('coaster_db_v2.csv')

# Define a function to extract the name from the coaster_name
def extract_name(coaster_name):
    if isinstance(coaster_name, str) and '(' in coaster_name:
        return coaster_name.split(' (')[0]
    else:
        return coaster_name

# Define a function to check if coaster name is similar
def similar_name(name1, name2):
    if isinstance(name1, float) or isinstance(name2, float):
        return False
    return str(name1).lower() == str(name2).lower()

# Combine the two dataframes
merged_df = pd.merge(df1, df2, left_on=['Name', 'Amusement Park'], right_on=['coaster_name', 'Location'], how='outer')


# Filter rows based on similar name and location
if 'Name' in merged_df.columns:
    filtered_df = merged_df[
        merged_df.apply(lambda row: similar_name(extract_name(row['Name']), extract_name(row['coaster_name'])), axis=1)
    ]
else:
    filtered_df = merged_df[
        merged_df.apply(lambda row: similar_name(extract_name(row['coaster_name']), extract_name(row['coaster_name'])), axis=1)
    ]

# Print the number of merged rows
print("Number of merged rows:", len(filtered_df))

# Save the filtered DataFrame to a new CSV file
filtered_df.to_csv('filtered_coaster_db.csv', index=False)


Number of merged rows: 1119


In [ ]:
# Load the DataFrame
df = pd.read_csv('filtered_coaster_db.csv')

# List of columns to drop
columns_to_drop = [
    'Coordinates_1', 'Coordinates_2', 'Status_1', 'Status_2', 'Opening date_1', 'Opening date_2',
    'Coordinates_3', 'Coordinates_4', 'Status_3', 'Status_4', 'Opening date_3', 'Opening date_4',
    'Flash Pass Available', 'Closing date_1', 'Closing date_2', 'Closing date_3', 'Closing date_4',
    'Fast Lane available', 'Location_1', 'Location_2', 'Location_3', 'Location_4', 'Must transfer from wheelchair',
    'Theme', 'Single rider line available', 'Restraint Style', 'Coordinates_5', 'Coordinates_6',
    'Status_5', 'Status_6', 'Opening date_5', 'Opening date_6', 'Coordinates_7', 'Coordinates_8',
    'Coordinates_9', 'Status_7', 'Status_8', 'Status_9', 'Opening date_7', 'Opening date_8',
    'Opening date_9', 'Park section_1', 'Park section_2', 'Park section_3', 'Park section_4',
    'Coordinates_10', 'Status_10', 'Opening date_10', 'Replaced by_1', 'Replaced by_2', 'Coordinates_11',
    'Coordinates_12', 'Coordinates_13', 'Coordinates_14', 'Coordinates_15', 'Status_11', 'Status_12',
    'Status_13', 'Status_14', 'Status_15', 'Opening date_11', 'Opening date_12', 'Opening date_13',
    'Opening date_14', 'Opening date_15', 'Closing date_5', 'Closing date_6', 'Closing date_7',
    'Closing date_8', 'Replaced by_3', 'Replaced by_4', 'Coordinates_16', 'Coordinates_17', 'Coordinates_18',
    'Coordinates_19', 'Coordinates_20', 'Status_16', 'Status_17', 'Status_18', 'Status_19', 'Status_20',
    'Opening date_16', 'Opening date_17', 'Opening date_18', 'Opening date_19', 'Opening date_20',
    'Closing date_9', 'Closing date_10', 'Closing date_11', 'Closing date_12', 'Coordinates_21',
    'Coordinates_22', 'Coordinates_23', 'Coordinates_24', 'Coordinates_25', 'Status_21', 'Status_22',
    'Status_23', 'Status_24', 'Status_25', 'Opening date_21', 'Opening date_22', 'Opening date_23',
    'Opening date_24', 'Opening date_25', 'Closing date_13', 'Closing date_14', 'Closing date_15',
    'Closing date_16', 'Coordinates_26', 'Coordinates_27', 'Coordinates_28', 'Coordinates_29', 'Coordinates_30',
    'Status_26', 'Status_27', 'Status_28', 'Status_29', 'Status_30', 'Opening date_26', 'Opening date_27',
    'Opening date_28', 'Opening date_29', 'Opening date_30', 'Closing date_17', 'Closing date_18', 'Closing date_19',
    'Closing date_20', 'Coordinates_31', 'Coordinates_32', 'Coordinates_33', 'Coordinates_34', 'Coordinates_35',
    'Coordinates_36', 'Status_31', 'Status_32', 'Status_33', 'Status_34', 'Status_35', 'Status_36',
    'Opening date_31', 'Opening date_32', 'Opening date_33', 'Opening date_34', 'Opening date_35', 'Opening date_36',
    'Closing date_21', 'Closing date_22', 'Closing date_23', 'Closing date_24', 'Closing date_25', 'Name_y',
    'Cost_1', 'Cost_2', 'Cost_3', 'Cost_4', 'Flash Pass available', 'Soft opening date', 'Park section_5',
    'Park section_6', 'Replaced by_5', 'Replaced by_6', 'Park section_7', 'Park section_8', 'Park section_9',
    'Replaced by_7', 'Replaced by_8', 'Replaced by_9', 'Acceleration', 'Park section_10', 'Park section_11',
    'Park section_12', 'Park section_13', 'Park section_14', 'Park section_15', 'Park section_16', 'Restraints',
    'Name_1', 'Name_2', 'Name_3', 'Name_4', 'Name_5', 'Name_6', 'Name_7', 'Name_8', 'Name_9', 
    'Replaced_1', 'Replaced_2', 'Replaced_3', 'Replaced_4', 'Fastrack available', 'Soft opening date_1', 
    'Soft opening date_2', 'Soft opening date_3', 'Soft opening date_4', 'Park section_17', 'Park section_18', 
    'Park section_19', 'Park section_20', 'Park section_21', 'Park section_22', 'Park section_23', 
    'Park section_24', 'Park section_25', 'Park section_26', 'Park section_27', 'Park section_28', 
    'Park section_29', 'Park section_30', 'Park section_31', 'Park section_32', 'Park section_33', 
    'Park section_34', 'Park section_35', 'Park section_36', 'Website_1', 'Website_2', 'Website_3', 
    'Website_4', 'Cost_5', 'Cost_6', 'Cost_7', 'Cost_8', 'Cost_9','Coordinates',	'Status_y',	'Opening date',	'General statistics','Website','Cost','Unnamed: 0','Height restriction'

]

# Drop the specified columns
df.drop(columns=columns_to_drop, inplace=True)

# Remove duplicates
df.drop_duplicates(inplace=True)

# Save the modified DataFrame to a new CSV file
df.to_csv('filtered_data.csv', index=False)

In [ ]:
df = pd.read_excel('new_data.xlsx')
df.head()

,Name,Amusement Park,Type,Design,Status,Opened,City,Region,Country,Make,...,Capacity,G-force.1,coaster_name,History,Closing date,Inversions_y,Trains,Replaced,Replaced by,General Statistics
0,10 Inversion Roller Coaster,Chimelong Paradise,Steel,Sit Down,Operating,2006,Guangzhou,Guangdong,China,Intamin,...,"1,340 riders per hour",NaN,10 Inversion Roller Coaster,History section not found,NaN,10,Single train with 7 cars. Riders are arranged ...,NaN,NaN,NaN
1,Abyssus,Energylandia,SteelÂ â€“ Launched,Sit Down,Operating,2021,Zator,MaÅ‚opolskie,Poland,Vekoma,...,1000 riders per hour,4.5,Abyssus,"In September 2017, Energylandia applied for fu...",NaN,4,3 trains with 4 cars. Riders are arranged 2 ac...,NaN,NaN,NaN
2,Adrenaline Peak,Oaks Amusement Park,SteelÂ â€“ Euro-Fighter,Sit Down,Operating,2018,Portland,Oregon,United States,Gerstlauer,...,NaN,NaN,Adrenaline Peak,"On August 17, 2017, Oaks Amusement Park announ...",NaN,3,3 trains with a single car. Riders are arrange...,Looping Thunder,NaN,NaN
3,Adventure Express,Kings Island,SteelÂ â€“ Mine Train,Sit Down,Operating,1991,Mason,Ohio,United States,Arrow Dynamics,...,1600 riders per hour,3.8,Adventure Express,Kings Island approached Arrow Dynamics and one...,NaN,0,3 trains with 5 cars. Riders are arranged 2 ac...,NaN,NaN,NaN
4,Alpen Blitz,Six Flags Great Adventure,Steel,Sit Down,Removed,1976,Jackson,New Jersey,United States,Anton Schwarzkopf,...,NaN,1,Alpen Blitz,History section not found,1978,NaN,NaN,Jumbo Jet,Haunted Castle,NaN
